In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
from google.colab import files
import requests
import re
import pandas as pd
from tqdm.notebook import tqdm as tqdm
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support import expected_conditions as EC
from ipywidgets import widgets, IntProgress, interact
from IPython.display import display, clear_output
import datetime
!pip install pytz
import pytz


date_today=datetime.datetime.strftime(datetime.datetime.now(pytz.timezone('Asia/Kolkata')),"%Y-%m-%d")
file_name = 'amzn '+ date_today + '.xlsx'

range_list=[(0,150),(150,250),(250,500),(500,"")]

page = "https://www.amazon.in/gp/browse.html?node=1380374031&ref_=nav_em_0_2_11_8_sbc_hk_decor"

def look_for_item(search_item,page,driver):
    driver.get(page)
    elem=driver.find_element_by_id("twotabsearchtextbox")
    elem.send_keys(search_item)
    elem.send_keys(Keys.ENTER)

def assert_nonprime(driver):
    clear_elements = driver.find_elements_by_class_name("a-link-normal.s-navigation-item.s-navigation-clear-link")
    if len(clear_elements)!=0:
        driver.find_element_by_class_name("a-link-normal.s-navigation-item.s-navigation-clear-link").click()

def set_range(low,high,driver):
    driver.find_element_by_id("low-price").clear()
    driver.find_element_by_id("low-price").send_keys(low)
    
    driver.find_element_by_id("high-price").clear()        
    driver.find_element_by_id("high-price").send_keys(high)
    driver.find_element_by_class_name('a-button-input').click()

def number_of_results(driver):
    text=driver.find_element_by_class_name('sg-col-inner').text
    num_results=text.split("result",maxsplit=1)[0].strip().split(" ",maxsplit=-1)[-1].strip()
    num_results=''.join([x for x in num_results if x.isdigit()==True])
    try:
        num_results=int(num_results)
    except ValueError as err:
        num_results=0
    return num_results

def switch_to_prime(driver):
    driver.find_element_by_class_name("a-link-normal.s-navigation-item").click()


item_widget=widgets.Text(description='Type here')

add_button = widgets.Button(description="Add product",
                        button_style='info',
                        tooltip='Click me',
                        icon='check')

output = widgets.Output()


data=pd.DataFrame()
products=list()


flip_range_list=["",('Min', 150), (150, 250), (250, 500), (500, 'Max')]


## Add flipkart funcs here-
### FLIPKART
def create_url(query,range_item,flip_ass_bin=0,augment=0):
    host              = "https://www.flipkart.com/search?q="
    query             =  query.replace(" ","+")
    pipe              =  "&p%5B%5D"
    low_range_prefix  = "=facets.price_range.from%3D"
    high_range_prefix = "=facets.price_range.to%3D"
    flip_assured      = "=facets.fulfilled_by%255B%255D%3DFlipkart%2BAssured"
    dom_tracker   = "&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off"
    url               = host + query + dom_tracker
    if augment==1:
        url           = url + "&augment=false"
    if range_item=='':
        low,high='','',
    if range_item!='':
        url= url  + pipe + low_range_prefix + str(range_item[0]) + pipe + high_range_prefix + str(range_item[1])
    if flip_ass_bin==1:
        url=url+pipe+flip_assured


    return url
def search_txt_corpus(response_text):
    if "Sorry, no results found!" in response_text:
        result=0
    else:
        regex_keyword = "(&quot;<span>+)(.*)(</span>&quot;+)"
        regex_keyword  = regex_keyword.lower()
        response_text = response_text.lower()
        end_idx       = re.search(regex_keyword,response_text).span()[0]
        result        = response_text[:end_idx].split(">")[-1].split("result",maxsplit=1)[0].strip().split(" ")[-1].strip()
        result        = int(''.join([x for x in result if x.isdigit()==True]))
    return result
def name_column(range_item,flip_status):
    if range_item=='':
        colname = flip_status+' Results'
    else:
        low, high=range_item[0],range_item[1]
        if low=="Min":
            low=0
        if high=='Max':
            high=''
        colname= flip_status + ' Results in Range '+str(low)+'-'+str(high)
    return colname

##### FLIPkart code complete


##  -- flipkart --


def on_button_clicked(b):
    with output:        
        print("------------")
        product_name = item_widget.value

        if len(product_name)<1:
            print("Not added, type something and click on button again")
        else:
            print(product_name, " - Added")
            products.append(product_name)
add_button.on_click(on_button_clicked)

review_button = widgets.Button(description="Review Products",
                               button_style='primary',
                               tooltip='Click me',
                               icon='check'
                              )

review_output = widgets.Output()

def review_button_clicked(b,products=products):
    with review_output:
        if len(products)>0:
            clear_output()
            products= list(set(products))
            for p, product in enumerate(products):
                print("Product number-",p+1,"-",product)
        else:
            print("No Items added yet")
            
review_button.on_click(review_button_clicked)


confirm_button = widgets.Button(description="Confirm & Begin",
                                button_style='info',
                                tooltip='Click me',
                                icon='check')

confirm_output = widgets.Output()



def confirm_button_clicked(b):
    with confirm_output:
        driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
        driver.get(page)

        data=pd.DataFrame()
        print("STEP 1 : Amazon")
        with tqdm(total=len(products)) as pbar_amzn:
            for s, search_item in enumerate(products):
                look_for_item(search_item,page,driver)
                driver.implicitly_wait(5)
                total_all    = number_of_results(driver=driver)
                switch_to_prime(driver)
                total_prime  =  number_of_results(driver=driver)

                data.loc[s,'Item']                = search_item
                data.loc[s,'AMZN Results']        = total_all
                data.loc[s,'AMZN Results (P)']    = total_prime

                for r, RANGE in enumerate(range_list):
                    low, high = str(RANGE[0]), str(RANGE[1])
                    assert_nonprime(driver)
                    set_range(low,high,driver=driver)

                    num_all = number_of_results(driver=driver)
                    range_string_1 ="AMZN Results in Range "+str(low)+"-"+str(high)
                    data.loc[s,range_string_1] = num_all

                    switch_to_prime(driver)
                    num_prime = number_of_results(driver=driver)                
                    range_string_2 ="AMZN (P) Results in Range "+str(low)+"-"+str(high)
                    data.loc[s,range_string_2] = num_prime
                pbar_amzn.update(1)
        print("STEP 2 : Flipkart")
        with tqdm(total=len(products)) as pbar_flip:
            for P, product_name in enumerate(products):
                augment=0
                for f, flip_status in enumerate(['Flipkart','Flipkart (FA)']):
                    for R, range_item in enumerate(flip_range_list):
        #url = create_url(product_name,range_item,flip_ass_bin=f)
                        for _ in range(5):
                            url = create_url(product_name,range_item,flip_ass_bin=f,augment=augment)
                            result=''
                            response           = requests.get(url)
                            response_text      = response.text
        # Assert no error- 1
                            if "Something's not right!" in response_text:
                                print("Retrying")
                                continue
                            else:
                                break
        # Assert no error- 2                
        #                    error_augment = re.findall("(Show results for +)(.*)( instead+)",response_text)

         #                   if len(error_augment)==0:

          #                      break
           #                 else:
            #                    print(error_augment[0])
        #                        url= url.replace("&as-show=on&as=off","&as-show=on&as=off&augment=false")
             #                   augment=1
              #                  continue
                        try:
                            result = search_txt_corpus(response_text)
                            colstr = name_column(range_item,flip_status)

                        except:
                            print(sys.exc_info())
                            print(product_name)
                            result="Check Manually"

                        data.loc[data.Item==product_name,colstr] = result

                pbar_flip.update(1)
            
            
        data.loc[:,'data_date']=date_today
        ordered_columns=['data_date','Item',
                         'AMZN Results','Flipkart Results',
                         'AMZN Results (P)', 'Flipkart (FA) Results',
                        'AMZN Results in Range 0-150','Flipkart Results in Range 0-150',
                        'AMZN (P) Results in Range 0-150','Flipkart (FA) Results in Range 0-150',
                        'AMZN Results in Range 150-250', 'Flipkart Results in Range 150-250',
                        'AMZN (P) Results in Range 150-250','Flipkart (FA) Results in Range 150-250',
                        'AMZN Results in Range 250-500', 'Flipkart Results in Range 250-500',
                        'AMZN (P) Results in Range 250-500','Flipkart (FA) Results in Range 250-500',
                        'AMZN Results in Range 500-','Flipkart Results in Range 500-',
                        'AMZN (P) Results in Range 500-','Flipkart (FA) Results in Range 500-']        
        data=data[ordered_columns]
        data.to_excel(file_name,index=False)

        print("Wait.. Preparing File")

        time.sleep(15)
        

        print("Look in the Files folder now")

confirm_button.on_click(confirm_button_clicked)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromi

In [ ]:
display(widgets.VBox([widgets.HBox([item_widget,add_button]),output]))

In [ ]:
display(widgets.VBox([review_button,review_output]))

In [ ]:
display(widgets.VBox([confirm_button,confirm_output]))